# Azure AI Agents με Υποστήριξη Πρωτοκόλλου Πλαισίου Μοντέλου (MCP) - Python

Αυτό το σημειωματάριο δείχνει πώς να χρησιμοποιήσετε τους Azure AI Agents με εργαλεία Πρωτοκόλλου Πλαισίου Μοντέλου (MCP) στην Python. Παρουσιάζει πώς να δημιουργήσετε έναν έξυπνο πράκτορα που μπορεί να αξιοποιήσει εξωτερικούς MCP servers (όπως το Microsoft Learn) για ενισχυμένες δυνατότητες χρησιμοποιώντας αυθεντικοποίηση χωρίς κλειδιά.


## Εγκατάσταση Απαραίτητων Πακέτων Python

Αρχικά, πρέπει να εγκαταστήσουμε τα απαραίτητα πακέτα Python:
- **azure-ai-projects**: Βασικό SDK για τα Azure AI Projects
- **azure-ai-agents**: SDK του Azure AI Agents για τη δημιουργία και διαχείριση πρακτόρων
- **azure-identity**: Παρέχει αυθεντικοποίηση χωρίς κλειδιά χρησιμοποιώντας το DefaultAzureCredential
- **mcp**: Υλοποίηση του Model Context Protocol για Python


## Οφέλη της Αυθεντικοποίησης Χωρίς Κλειδιά

Αυτό το notebook παρουσιάζει την **αυθεντικοποίηση χωρίς κλειδιά**, η οποία προσφέρει αρκετά πλεονεκτήματα:
- ✅ **Χωρίς διαχείριση API keys** - Χρησιμοποιεί αυθεντικοποίηση βασισμένη στην ταυτότητα του Azure
- ✅ **Ενισχυμένη ασφάλεια** - Δεν αποθηκεύονται μυστικά στον κώδικα ή στα αρχεία ρυθμίσεων
- ✅ **Αυτόματη ανανέωση διαπιστευτηρίων** - Το Azure διαχειρίζεται τον κύκλο ζωής των διαπιστευτηρίων
- ✅ **Έλεγχος πρόσβασης βάσει ρόλων** - Χρησιμοποιεί το Azure RBAC για λεπτομερή δικαιώματα
- ✅ **Υποστήριξη πολλαπλών περιβαλλόντων** - Λειτουργεί απρόσκοπτα σε περιβάλλον ανάπτυξης και παραγωγής

Το `DefaultAzureCredential` επιλέγει αυτόματα την καλύτερη διαθέσιμη πηγή διαπιστευτηρίων:
1. **Managed Identity** (όταν εκτελείται στο Azure)
2. **Διαπιστευτήρια Azure CLI** (κατά την τοπική ανάπτυξη)
3. **Διαπιστευτήρια Visual Studio**
4. **Μεταβλητές περιβάλλοντος** (αν έχουν ρυθμιστεί)
5. **Αυθεντικοποίηση μέσω διαδραστικού προγράμματος περιήγησης** (ως εναλλακτική λύση)


## Ρύθμιση Αυθεντικοποίησης Χωρίς Κλειδιά

**Προαπαιτούμενα για αυθεντικοποίηση χωρίς κλειδιά:**

### Για Τοπική Ανάπτυξη:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### Για Περιβάλλοντα Azure:
- Ενεργοποιήστε **Διαχειριζόμενη Ταυτότητα που έχει εκχωρηθεί από το σύστημα** στον πόρο Azure σας
- Εκχωρήστε κατάλληλους **ρόλους RBAC** στη διαχειριζόμενη ταυτότητα:
  - `Cognitive Services OpenAI User` για πρόσβαση στο Azure OpenAI
  - `AI Developer` για πρόσβαση σε έργα Azure AI

### Μεταβλητές Περιβάλλοντος (Προαιρετικά):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**Δεν χρειάζονται API keys ή strings σύνδεσης!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## Εισαγωγή Απαραίτητων Βιβλιοθηκών

Εισάγετε τις απαραίτητες Python βιβλιοθήκες:
- **os, time**: Βασικές βιβλιοθήκες της Python για μεταβλητές περιβάλλοντος και καθυστερήσεις
- **AIProjectClient**: Κύριος πελάτης για τα Azure AI Projects
- **DefaultAzureCredential**: Αυθεντικοποίηση χωρίς κλειδιά για τις υπηρεσίες Azure
- **MCP-related classes**: Για τη δημιουργία και διαχείριση εργαλείων MCP και την επεξεργασία εγκρίσεων


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## Ρύθμιση Παραμέτρων Διακομιστή MCP

Ρυθμίστε τη διαμόρφωση του διακομιστή MCP χρησιμοποιώντας μεταβλητές περιβάλλοντος με προεπιλεγμένες τιμές:
- **MCP_SERVER_URL**: Η διεύθυνση URL του διακομιστή MCP (προεπιλογή: Microsoft Learn API)
- **MCP_SERVER_LABEL**: Μια ετικέτα για την αναγνώριση του διακομιστή MCP (προεπιλογή: "mslearn")

Αυτή η προσέγγιση επιτρέπει ευέλικτη διαμόρφωση σε διαφορετικά περιβάλλοντα.


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Δημιουργία Πελάτη Έργου Azure AI (Χωρίς Κλειδιά)

Αρχικοποιήστε τον πελάτη έργου Azure AI χρησιμοποιώντας **αυθεντικοποίηση χωρίς κλειδιά**:
- **endpoint**: Η διεύθυνση URL του τελικού σημείου του έργου Azure AI Foundry
- **credential**: Χρησιμοποιεί `DefaultAzureCredential()` για ασφαλή αυθεντικοποίηση χωρίς κλειδιά
- **Δεν απαιτούνται κλειδιά API**: Ανακαλύπτει και χρησιμοποιεί αυτόματα την καλύτερη διαθέσιμη διαπίστευση

**Ροή Αυθεντικοποίησης:**
1. Ελέγχει για Managed Identity (σε περιβάλλοντα Azure)
2. Επιστρέφει σε διαπιστεύσεις Azure CLI (για τοπική ανάπτυξη)
3. Χρησιμοποιεί άλλες διαθέσιμες πηγές διαπιστεύσεων, εάν χρειάζεται

Αυτή η προσέγγιση εξαλείφει την ανάγκη διαχείρισης κλειδιών API ή συμβολοσειρών σύνδεσης στον κώδικά σας.


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## Δημιουργία Ορισμού Εργαλείου MCP

Δημιουργήστε ένα εργαλείο MCP που συνδέεται με τον διακομιστή MCP του Microsoft Learn:
- **server_label**: Αναγνωριστικό για τον διακομιστή MCP
- **server_url**: URL σημείο τερματισμού του διακομιστή MCP
- **allowed_tools**: Προαιρετική λίστα για περιορισμό των εργαλείων που μπορούν να χρησιμοποιηθούν (κενή λίστα επιτρέπει όλα τα εργαλεία)

Αυτό το εργαλείο θα επιτρέψει στον πράκτορα να έχει πρόσβαση στην τεκμηρίωση και τους πόρους του Microsoft Learn.


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## Δημιουργία Πράκτορα και Εκτέλεση Συνομιλίας (Ροή Εργασίας Χωρίς Κλειδιά)

Αυτή η ολοκληρωμένη ενότητα παρουσιάζει τη συνολική **ροή εργασίας πράκτορα χωρίς κλειδιά**:

1. **Δημιουργία Πράκτορα AI**: Ρυθμίστε έναν πράκτορα με το μοντέλο GPT-4.1 nano και τα εργαλεία MCP
2. **Δημιουργία Νήματος**: Δημιουργήστε ένα νήμα συνομιλίας για επικοινωνία
3. **Αποστολή Μηνύματος**: Ρωτήστε τον πράκτορα για τις διαφορές μεταξύ Azure OpenAI και OpenAI
4. **Διαχείριση Εγκρίσεων Εργαλείων**: Εγκρίνετε αυτόματα τις κλήσεις εργαλείων MCP όταν απαιτείται
5. **Παρακολούθηση Εκτέλεσης**: Παρακολουθήστε την πρόοδο του πράκτορα και διαχειριστείτε τυχόν απαιτούμενες ενέργειες
6. **Εμφάνιση Αποτελεσμάτων**: Εμφανίστε τη συνομιλία και τις λεπτομέρειες χρήσης εργαλείων

**Χαρακτηριστικά Χωρίς Κλειδιά:**
- ✅ **Χωρίς ενσωματωμένα μυστικά** - Όλος ο έλεγχος ταυτότητας γίνεται μέσω της ταυτότητας Azure
- ✅ **Ασφαλές από προεπιλογή** - Χρησιμοποιεί έλεγχο πρόσβασης βάσει ρόλων
- ✅ **Απλοποιημένη ανάπτυξη** - Δεν απαιτείται διαχείριση διαπιστευτηρίων
- ✅ **Φιλικό προς έλεγχο** - Όλη η πρόσβαση παρακολουθείται μέσω της ταυτότητας Azure

Ο πράκτορας θα χρησιμοποιήσει τα εργαλεία MCP για πρόσβαση σε πόρους του Microsoft Learn με πλήρη ασφάλεια και χωρίς διαχείριση API κλειδιών.


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**Αποποίηση ευθύνης**:  
Αυτό το έγγραφο έχει μεταφραστεί χρησιμοποιώντας την υπηρεσία αυτόματης μετάφρασης [Co-op Translator](https://github.com/Azure/co-op-translator). Παρόλο που καταβάλλουμε προσπάθειες για ακρίβεια, παρακαλούμε να έχετε υπόψη ότι οι αυτοματοποιημένες μεταφράσεις ενδέχεται να περιέχουν σφάλματα ή ανακρίβειες. Το πρωτότυπο έγγραφο στη μητρική του γλώσσα θα πρέπει να θεωρείται η αυθεντική πηγή. Για κρίσιμες πληροφορίες, συνιστάται επαγγελματική ανθρώπινη μετάφραση. Δεν φέρουμε ευθύνη για τυχόν παρεξηγήσεις ή εσφαλμένες ερμηνείες που προκύπτουν από τη χρήση αυτής της μετάφρασης.
